In [3]:
using DMUStudent.HW3: HW3, DenseGridWorld, visualize_tree
using POMDPs: actions, @gen, isterminal, discount, statetype, actiontype, simulate, states, initialstate
using D3Trees: inchrome
using StaticArrays: SA
using Statistics: mean, std, mean

In [13]:
##############
# Instructions
##############
#=

This starter code is here to show examples of how to use the HW3 code that you
can copy and paste into your homework code if you wish. It is not meant to be a
fill-in-the blank skeleton code, so the structure of your final submission may
differ from this considerably.

Please make sure to update DMUStudent to gain access to the HW3 module.

=#

############
# Question 2
############


mdp = HW3.DenseGridWorld(seed = 3)
a = actions(mdp)
s = states(mdp)
policy_function = rand(actions(mdp))
start_state = SA[19, 19]

function rollout(mdp, start_state, max_steps=100)
    r_total = 0.0
    a = actions(mdp)
    s = start_state
    t = 0
    while !isterminal(mdp, s) && t < max_steps
        a = rand(actions(mdp))
        s, r = @gen(:sp,:r)(mdp, s, a)
        r_total += discount(mdp)^t*r
        t += 1
    end
    return r_total # replace this with the reward
end


function heuristic_policy(s)
    xtrue = s[1]
    ytrue = s[2]
    
    x = mod(s[1],20)
    y = mod(s[2],20)
    
    # If between bounds
    if (20 - x > 10) & (20 - x != 20) 
        a = :left
    elseif (20 - x <= 10) & (20 - x != 0)
        a = :right
    elseif 20 - y > 10
        a = :down
    else 
        a = :up
    end
    
    # # If outside of internal square 
    # if 60 - xtrue > 40 
    #     a = :right
    # elseif 60 - xtrue < 20
    #     a = :left 
    # elseif 60 - ytrue > 40
    #     a = :up
    # elseif 60 - ytrue < 20
    #     a = :down
    # else
    #     return a
    # end
    
    return a 
end


function rollout_smart(mdp, start_state, heuristic_policy, max_steps=100)
    r_total = 0.0
    a = actions(mdp)
    s = start_state
    t = 0
    while !isterminal(mdp, s) && t < max_steps
        a = heuristic_policy(s)
        s, r = @gen(:sp,:r)(mdp, s, a)
        r_total += discount(mdp)^t*r
        t += 1
    end
    return r_total # replace this with the reward
end



# # This code runs monte carlo simulations: you can calculate the mean and standard error from the results
# results_random = [rollout(mdp, rand(initialstate(mdp))) for _ in 1:10000]
# mean_random = mean(results_random)
# SEM_random = std(results_random)/(length(results_random)^0.5)
# @show mean_random
# @show SEM_random

# results_smart = [rollout_smart(mdp, rand(initialstate(mdp)), heuristic_policy) for _ in 1:10000]
# mean_smart = mean(results_smart)
# SEM_smart = std(results_smart)/(length(results_smart)^0.5)

# @show mean_smart
# @show SEM_smart

# @show difference = mean_smart - mean_random

heuristic_policy([19,20])

:right

In [3]:
m = DenseGridWorld()

S = statetype(m)
A = actiontype(m)

# These would be appropriate containers for your Q, N, and t dictionaries:
n = Dict{Tuple{S, A}, Int}()
q = Dict{Tuple{S, A}, Float64}()
t = Dict{Tuple{S, A, S}, Int}()

# This is an example state - it is a StaticArrays.SVector{2, Int}
s = SA[19,19]
@show typeof(s)
@assert s isa statetype(m)

# here is an example of how to visualize a dummy tree (q, n, and t should actually be filled in your mcts code, but for this we fill it manually)
q[(SA[1,1], :right)] = 0.0
q[(SA[2,1], :right)] = 0.0
n[(SA[1,1], :right)] = 1
n[(SA[2,1], :right)] = 0
t[(SA[1,1], :right, SA[2,1])] = 1

q[[1,2],:right] = 1
q[s,:right] = 2



typeof(s) = StaticArraysCore.SVector{2, Int64}


2

In [5]:
mdp = DenseGridWorld(seed=4)

discount(mdp)

0.95

In [15]:
############
# Question 3
############

##########################################
function rollout_smart(mdp, start_state, heuristic_policy, max_steps=100)
    r_total = 0.0
    a = actions(mdp)
    s = start_state
    t = 0
    while !isterminal(mdp, s) && t < max_steps
        a = heuristic_policy(s)
        s, r = @gen(:sp,:r)(mdp, s, a)
        r_total += discount(mdp)^t*r
        t += 1
    end
    return r_total # replace this with the reward
end
##########################################
function bonus(N, Ns, s, a)
    
    if N[(s,a)] == 0 || Ns == 0
        return Inf
    end    
    
    return sqrt(log(Ns)/N[(s,a)])  
end


##########################################

function explore(mdp,N,s,Q,c)
    A = actions(mdp)
    Ns = sum(N[(s,a)] for a in A)
    
    index = argmax(Q[(s,a)] + c * bonus(N,Ns,s,a) for a in A)
    return A[index] 
end

##########################################
function simulateMCTS(mdp,s,N,Q,d,c,V,T,discount)
    A = actions(mdp)
    
    if d <= 0 
        return rollout_smart(mdp, s, heuristic_policy), N, T, Q
    end
    
    
    if !haskey(N, (s,first(A)) )

        for a in A
            N[s,a] = 0
            Q[s,a] = 0
        end
        return rollout_smart(mdp, s, heuristic_policy), N, T, Q
    end
    
    a = explore(mdp,N,s,Q,c)
    sprime , r = @gen(:sp,:r)(mdp,s,a)
    
    if !haskey(T, (s,a,sprime) )
        T[s,a,sprime] = 0
    end
    
    T[(s,a,sprime)] += 1
    
    q, N, T, Q = simulateMCTS(mdp,sprime,N,Q,d-1,c,V,T,discount)         
    q = r + discount * q
    
    N[s,a] += 1
    Q[s,a] += (q - Q[(s,a)]) / N[(s,a)]
    return q, N, T, Q
    
end

        
##########################################
function MCTS(mdp, reps)
    s = SA[19,19]
    dis = discount(mdp)
    d = 15
    c = 120
    V = 0
    S = statetype(mdp)
    A = actiontype(mdp)
    N = Dict{Tuple{S, A}, Int}()
    Q = Dict{Tuple{S, A}, Float64}()
    T = Dict{Tuple{S, A, S}, Int}()
    A = actions(mdp)
    
    
    
    for i in 1:reps 
        q, N, T, Q = simulateMCTS(mdp,s,N,Q,d,c,V,T,dis)
    end
    

    index = argmax(Q[s,a] for a in actions(mdp))
        
    return Q, N, T, A[index]

end

mdp = DenseGridWorld(seed=4)

Q,N,T,A = MCTS(mdp, 7)



inchrome(visualize_tree(Q, N, T, SA[19,19]))

Process(`open -a 'Google Chrome' /var/folders/0g/ynx547mn6zqdsjcn17zrpwtc0000gn/T/jl_Zl7vXf/tree.html`, ProcessExited(0))

In [18]:
############
# Question 4
############

##########################################
function rollout_smart(mdp, start_state, heuristic_policy, max_steps=100)
    r_total = 0.0
    a = actions(mdp)
    s = start_state
    t = 0
    while !isterminal(mdp, s) && t < max_steps
        a = heuristic_policy(s)
        s, r = @gen(:sp,:r)(mdp, s, a)
        r_total += discount(mdp)^t*r
        t += 1
    end
    return r_total # replace this with the reward
end

##########################################
function rollout_MCTS(mdp, start_state, max_steps=100)
    r_total = 0.0
    a = actions(mdp)
    s = start_state
    t = 0
    while !isterminal(mdp, s) && t < max_steps
        _,_,_,a = MCTS(mdp,s)
        s, r = @gen(:sp,:r)(mdp, s, a)
        r_total += discount(mdp)^t*r
        t += 1
    end
    return r_total # replace this with the reward
end
##########################################
function bonus(N, Ns, s, a)
    
    if N[(s,a)] == 0 || Ns == 0
        return Inf
    end    
    
    return sqrt(log(Ns)/N[(s,a)])  
end


##########################################

function explore(mdp,N,s,Q,c)
    A = actions(mdp)
    Ns = sum(N[(s,a)] for a in A)
    
    index = argmax(Q[(s,a)] + c * bonus(N,Ns,s,a) for a in A)
    return A[index] 
end

##########################################
function simulateMCTS(mdp,s,N,Q,d,c,V,T,discount)
    A = actions(mdp)
    
    if d <= 0 
        return rollout_smart(mdp, s, heuristic_policy), N, T, Q
    end
    
    
    if !haskey(N, (s,first(A)) )

        for a in A
            N[s,a] = 0
            Q[s,a] = 0
        end
        return rollout_smart(mdp, s, heuristic_policy), N, T, Q
    end
    
    a = explore(mdp,N,s,Q,c)
    sprime , r = @gen(:sp,:r)(mdp,s,a)
    
    if !haskey(T, (s,a,sprime) )
        T[s,a,sprime] = 0
    end
    
    T[(s,a,sprime)] += 1
    
    q, N, T, Q = simulateMCTS(mdp,sprime,N,Q,d-1,c,V,T,discount)         
    q = r + discount * q
    
    N[s,a] += 1
    Q[s,a] += (q - Q[(s,a)]) / N[(s,a)]
    return q, N, T, Q
    
end

        
##########################################
function MCTS(mdp, s)
    start = time_ns()
    dis = discount(mdp)
    d = 15
    c = 120
    V = 0
    S = statetype(mdp)
    A = actiontype(mdp)
    N = Dict{Tuple{S, A}, Int}()
    Q = Dict{Tuple{S, A}, Float64}()
    T = Dict{Tuple{S, A, S}, Int}()
    A = actions(mdp)
    
    
    count = 0
    while time_ns() < start + 40_000_000 
        q, N, T, Q = simulateMCTS(mdp,s,N,Q,d,c,V,T,dis)
        count += 1
    end
    @show count

    index = argmax(Q[s,a] for a in actions(mdp))
        
    return Q, N, T, A[index]

end

mdp = DenseGridWorld(seed=4)

# Q,N,T,A = MCTS(mdp, 7)


# This code runs monte carlo simulations: you can calculate the mean and standard error from the results
results_random = [rollout(mdp, rand(initialstate(mdp))) for _ in 1:10000]
mean_random = mean(results_random)
SEM_random = std(results_random)/(length(results_random)^0.5)
@show mean_random
@show SEM_random

results_smart = [rollout_MCTS(mdp, rand(initialstate(mdp))) for _ in 1:100]
mean_smart = mean(results_smart)
SEM_smart = std(results_smart)/(length(results_smart)^0.5)
@show mean_smart
@show SEM_smart


@show difference = mean_smart - mean_random

mean_random = -94.42576069587611
SEM_random = 0.7960362875655713
count = 1
count = 7084
count = 4955
count = 6827
count = 6663
count = 5120
count = 7034
count = 7419
count = 6094
count = 7390
count = 7066
count = 6290
count = 7649
count = 8244
count = 8086
count = 7730
count = 9208
count = 9455
count = 8842
count = 8962
count = 9246
count = 10240
count = 10018
count = 8337
count = 6023
count = 6998
count = 8302
count = 8507
count = 7260
count = 9025
count = 9146
count = 9013
count = 9724
count = 10331
count = 8247
count = 6191
count = 7819
count = 7364
count = 6536
count = 7618
count = 7567
count = 6759
count = 7726
count = 7273
count = 6184
count = 7459
count = 8081
count = 6994
count = 8496
count = 8310
count = 8600
count = 8024
count = 8657
count = 8952
count = 9385
count = 9338
count = 9049
count = 9521
count = 9633
count = 10371
count = 8255
count = 7577
count = 9593
count = 9881
count = 9889
count = 10110
count = 10317
count = 9673
count = 7760
count = 6853
count = 7920
count = 7

count = 8064
count = 6883
count = 7908
count = 7838
count = 7851
count = 6493
count = 7916
count = 8156
count = 9070
count = 9115
count = 9008
count = 9112
count = 7605
count = 9481
count = 9644
count = 9957
count = 10208
count = 9579
count = 7414
count = 6416
count = 7729
count = 7625
count = 6376
count = 7709
count = 7461
count = 7929
count = 7795
count = 6405
count = 7622
count = 7454
count = 6390
count = 7669
count = 6539
count = 7362
count = 7788
count = 6540
count = 7447
count = 7603
count = 6671
count = 7199
count = 8507
count = 6808
count = 9048
count = 9147
count = 9265
count = 9154
count = 9318
count = 9552
count = 9979
count = 7910
count = 6412
count = 7262
count = 6306
count = 7252
count = 7146
count = 6377
count = 7289
count = 6294
count = 7130
count = 6937
count = 6953
count = 6806
count = 6124
count = 6877
count = 5910
count = 7569
count = 7521
count = 6636
count = 7616
count = 6923
count = 7740
count = 7764
count = 6139
count = 7548
count = 7421
count = 6312
count = 707

count = 8311
count = 8593
count = 7553
count = 8996
count = 9187
count = 9400
count = 9774
count = 9849
count = 10116
count = 8909
count = 9985
count = 9862
count = 10086
count = 10474
count = 8162
count = 8036
count = 6863
count = 8215
count = 8020
count = 6080
count = 7555
count = 6156
count = 6917
count = 5881
count = 6925
count = 7229
count = 6290
count = 7455
count = 6110
count = 7008
count = 6061
count = 7377
count = 6282
count = 7376
count = 7272
count = 6423
count = 7723
count = 6736
count = 8185
count = 8456
count = 7274
count = 8471
count = 8710
count = 8641
count = 8808
count = 8510
count = 9090
count = 9599
count = 9678
count = 9606
count = 9763
count = 9975
count = 10344
count = 9933
count = 7588
count = 8515
count = 8739
count = 8358
count = 7253
count = 8800
count = 8961
count = 9325
count = 10014
count = 9487
count = 6632
count = 7378
count = 7338
count = 6281
count = 7226
count = 6146
count = 7314
count = 7285
count = 6332
count = 7438
count = 6357
count = 7489
count =

100.33002467709173

In [81]:
############
# Question 5
############

##########################################
function rollout_smart(mdp, start_state, heuristic_policy, max_steps=100)
    r_total = 0.0
    a = actions(mdp)
    s = start_state
    t = 0
    while !isterminal(mdp, s) && t < max_steps
        a = heuristic_policy(s)
        s, r = @gen(:sp,:r)(mdp, s, a)
        r_total += discount(mdp)^t*r
        t += 1
    end
    return r_total # replace this with the reward
end

##########################################
function rollout_MCTS(mdp, start_state, max_steps=100)
    r_total = 0.0
    a = actions(mdp)
    s = start_state
    t = 0
    while !isterminal(mdp, s) && t < max_steps
        _,_,_,a = MCTS5(mdp,s)
        s, r = @gen(:sp,:r)(mdp, s, a)
        r_total += discount(mdp)^t*r
        t += 1
    end
    return r_total 
end
##########################################
function bonus(N, Ns, s, a)
    
    if N[(s,a)] == 0 || Ns == 0
        return Inf
    end    
    
    return sqrt(log(Ns)/N[(s,a)])  
end


##########################################

function explore(mdp,N,s,Q,c)
    A = actions(mdp)
    Ns = sum(N[(s,a)] for a in A)
    
    index = argmax(Q[(s,a)] + c * bonus(N,Ns,s,a) for a in A)
    return A[index] 
end

##########################################
function simulateMCTS(mdp,s,N,Q,d,c,V,T,discount)
    A = actions(mdp)
    
    if d <= 0 
        return rollout_smart(mdp, s, heuristic_policy), N, T, Q
    end
    
    
    if !haskey(N, (s,first(A)) )

        for a in A
            N[s,a] = 0
            Q[s,a] = 0
        end
        return rollout_smart(mdp, s, heuristic_policy), N, T, Q
    end
    
    a = explore(mdp,N,s,Q,c)
    sprime , r = @gen(:sp,:r)(mdp,s,a)
    
    if !haskey(T, (s,a,sprime) )
        T[s,a,sprime] = 0
    end
    
    T[(s,a,sprime)] += 1
    
    q, N, T, Q = simulateMCTS(mdp,sprime,N,Q,d-1,c,V,T,discount)         
    q = r + discount * q
    
    N[s,a] += 1
    Q[s,a] += (q - Q[(s,a)]) / N[(s,a)]
    return q, N, T, Q
    
end

        
##########################################
function MCTS5(mdp, s)
    start = time_ns()
    dis = discount(mdp)
    d = 15
    c = 120
    V = 0
    S = statetype(m)
    A = actiontype(m)
    N = Dict{Tuple{S, A}, Int}()
    Q = Dict{Tuple{S, A}, Float64}()
    T = Dict{Tuple{S, A, S}, Int}()
    A = actions(mdp)
    
    
    
    while time_ns() < start + 40_000_000 
        q, N, T, Q = simulateMCTS(mdp,s,N,Q,d,c,V,T,dis)
    end
    

    index = argmax(Q[s,a] for a in actions(mdp))
        
    return A[index]

end

MCTS5(mdp,SA[5,5])

HW3.evaluate(MCTS5, "julian.lemabulliard@colorado.edu")


Progress:  43%|█████████████████▋                       |  ETA: 0:00:50┌ Warning: Wall clock time limit of 0.05 seconds for choosing an action online exceeded. Taking a random action.
└ @ DMUStudent.HW3 none:14
Progress:  46%|██████████████████▉                      |  ETA: 0:00:49┌ Warning: Wall clock time limit of 0.05 seconds for choosing an action online exceeded. Taking a random action.
└ @ DMUStudent.HW3 none:14
Progress:  56%|███████████████████████                  |  ETA: 0:00:41┌ Warning: Wall clock time limit of 0.05 seconds for choosing an action online exceeded. Taking a random action.
└ @ DMUStudent.HW3 none:14
Progress:  63%|█████████████████████████▉               |  ETA: 0:00:36┌ Warning: Wall clock time limit of 0.05 seconds for choosing an action online exceeded. Taking a random action.
└ @ DMUStudent.HW3 none:14
Progress:  76%|███████████████████████████████▏         |  ETA: 0:00:22┌ Warning: Wall clock time limit of 0.05 seconds for choosing an action online exceed

(hw = 3.1, email = "julian.lemabulliard@colorado.edu", score = 64.0979875243398, hash = "9af0b6fffe02df0f49171229946744317faf79bd2d421106e59e6206f41ea13d")